In [1]:
# D:\FJTCM\DeepLife\DLProject\run_featurizer_lifespan.py

from __future__ import division
from __future__ import print_function
import time
import tensorflow as tf # Comment out if not strictly needed by deepchem features you use
import deepchem as dc
import rdkit
from rdkit import Chem
import numpy as np
import scipy.sparse as sp
# import networkx as nx
import os
import pickle
import pandas as pd
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.Chem import MolFromSmiles,MolToSmiles

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
No normalization for NumAmideBonds. Feature removed!
No normalization for NumAtomStereoCenters. Feature removed!
No normalization for NumBridgeheadAtoms. Feature removed!
No normalization for NumHeterocycles. Feature removed!
No normalization for NumSpiroAtoms. Feature removed!
No normalization for NumUnspecifiedAtomStereoCenters. Feature removed!
No normalization for Phi. Feature removed!


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


c:\ProgramData\anaconda3\envs\deepchem\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\ProgramData\anaconda3\envs\deepchem\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
def get_csv(path, smipos): # Original function
    df = pd.read_csv(path)
    df= df.rename(columns={smipos:'SMILES_NS'}) # This rename might not be needed for your new CSVs
    return df

def clean_smiles(smiles: pd.Series) -> pd.Series:
    if not isinstance(smiles, pd.Series):
        smiles = pd.Series(smiles)
    remover = SaltRemover()
    SMILES_desalt = []
    defect_indices = []
    original_indices = smiles.index.tolist() # Keep track of original indices

    for original_idx, smi_string in smiles.items(): # Iterate with original index
        mol = MolFromSmiles(smi_string)
        if mol is not None:
            try:
                mol_desalt = remover.StripMol(mol)
                mol_SMILES = MolToSmiles(mol_desalt)
                SMILES_desalt.append(mol_SMILES)
            except Exception as e:
                defect_indices.append(original_idx)
                SMILES_desalt.append(smi_string) # Keep original if desalting fails
        else:
            defect_indices.append(original_idx)
            SMILES_desalt.append(None) # Mark as None to be dropped later

    # print(f"Number of SMILES with cleaning defects or invalid: {len(defect_indices)}")
    # Create a new series with original index, then dropna
    # This helps in aligning with original DataFrame if needed before featurization steps
    cleaned_series = pd.Series(SMILES_desalt, index=original_indices).dropna()
    return cleaned_series


def preprocess_graph(data):
    if data is None or not isinstance(data, np.ndarray) or data.ndim != 2 or data.shape[0] != data.shape[1]:
        # print(f"Warning: Invalid data for preprocess_graph: {type(data)}. Returning None.")
        return None
    if data.shape[0] == 0: # Handle empty adjacency matrix
        return data # Or np.array([])
    adj_ = data + sp.eye(data.shape[0])
    rowsum = np.array(adj_.sum(1)).flatten() # Ensure rowsum is 1D
    rowsum[rowsum == 0] = 1e-9 # Avoid division by zero
    degree_mat_inv_sqrt = np.diag(np.power(rowsum, -0.5))
    adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt)
    return np.array(adj_normalized)

def smiles_get_features(smi_string, max_atoms=60):
    if pd.isna(smi_string): return np.nan
    mol = Chem.MolFromSmiles(smi_string)
    if mol is None: return np.nan
    if mol.GetNumAtoms() == 0 : return np.nan # Handle empty molecules
    if mol.GetNumAtoms() > max_atoms: return "EXCEED_MAX_ATOMS" # Special marker

    try:
        featurizer = dc.feat.ConvMolFeaturizer()
        features_obj = featurizer.featurize([mol])[0]
        if features_obj is None or isinstance(features_obj, bool): return np.nan # Deepchem can return False
        atom_features = features_obj.get_atom_features()
        return atom_features
    except Exception as e:
        # print(f"Error in smiles_get_features for {smi_string}: {e}")
        return np.nan

def smiles_get_adj(smi_string, max_atoms=60):
    if pd.isna(smi_string): return np.nan
    mol = Chem.MolFromSmiles(smi_string)
    if mol is None: return np.nan
    num_atoms = mol.GetNumAtoms()
    if num_atoms == 0 : return np.nan
    if num_atoms > max_atoms: return "EXCEED_MAX_ATOMS"

    try:
        # Using RDKit directly for adjacency is often more straightforward
        adj = Chem.GetAdjacencyMatrix(mol)
        return adj.astype(np.float32) # Ensure float
    except Exception as e:
        # print(f"Error in smiles_get_adj for {smi_string}: {e}")
        return np.nan

def sim_graph(smi_string, max_atoms=60): # For "graph.npy"
    if pd.isna(smi_string): return np.nan
    mol = Chem.MolFromSmiles(smi_string)
    if mol is None: return np.nan
    num_atoms = mol.GetNumAtoms()
    if num_atoms == 0 : return np.nan
    if num_atoms > max_atoms: return "EXCEED_MAX_ATOMS"
    try:
        Chem.Kekulize(mol)
        atoms = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
        am = Chem.GetAdjacencyMatrix(mol, useBO=True).astype(np.float32)
        for i, atom_val in enumerate(atoms):
            am[i, i] = atom_val
        return am
    except Exception as e:
        # print(f"Error in sim_graph for {smi_string}: {e}")
        return np.nan


def pad_array(arr, target_shape, constant_values=0):
    """Pads a 2D numpy array to a target shape."""
    if not isinstance(arr, np.ndarray):
        # print(f"Cannot pad non-array type: {type(arr)}")
        return np.full(target_shape, constant_values, dtype=np.float32) # Return a padded zero/const array

    pad_width = []
    for i in range(len(target_shape)):
        if i < arr.ndim:
            pad_width.append((0, max(0, target_shape[i] - arr.shape[i])))
        else: # Target shape has more dimensions
            pad_width.append((0, target_shape[i]))
    return np.pad(arr, pad_width, 'constant', constant_values=constant_values)


def featurize_and_save(smiles_series: pd.Series,
                        target_name_suffix: str, # e.g., "LifespanReg_train"
                        output_npdata_dir: str,
                        max_atoms_limit: int,
                        atom_feat_dim: int):
    """
    Generates and saves adj.npy, feature.npy, graph.npy.
    Returns a list of indices from the input smiles_series that were successfully processed.
    """
    print(f"Starting featurization for: {target_name_suffix}")
    os.makedirs(output_npdata_dir, exist_ok=True)

    all_adj_matrices = []
    all_atom_features = []
    all_sim_graphs = []
    successfully_processed_indices = [] # Original indices from smiles_series

    for original_idx, smi in smiles_series.items():
        adj = smiles_get_adj(smi, max_atoms_limit)
        feat = smiles_get_features(smi, max_atoms_limit)
        simg = sim_graph(smi, max_atoms_limit)

        # Check if any featurization step returned the special marker or NaN
        if isinstance(adj, str) or isinstance(feat, str) or isinstance(simg, str) or \
           pd.isna(adj).all() if isinstance(adj, np.ndarray) else pd.isna(adj) or \
           pd.isna(feat).all() if isinstance(feat, np.ndarray) else pd.isna(feat) or \
           pd.isna(simg).all() if isinstance(simg, np.ndarray) else pd.isna(simg):
            print(f"Skipping SMILES (index {original_idx}): {smi} due to featurization issue or exceeding max_atoms.")
            continue

        # Normalize adjacency matrix
        norm_adj = preprocess_graph(adj)
        if norm_adj is None:
            print(f"Skipping SMILES (index {original_idx}): {smi} due to normalization issue.")
            continue

        all_adj_matrices.append(norm_adj)
        all_atom_features.append(feat)
        all_sim_graphs.append(simg)
        successfully_processed_indices.append(original_idx)

    if not successfully_processed_indices:
        print(f"No SMILES were successfully featurized for {target_name_suffix}.")
        # Save empty arrays to prevent downstream errors if files are expected
        np.save(os.path.join(output_npdata_dir, "adj.npy"), np.array([]))
        np.save(os.path.join(output_npdata_dir, "feature.npy"), np.array([]))
        np.save(os.path.join(output_npdata_dir, "graph.npy"), np.array([]))
        return [] # Return empty list of indices

    # Pad and stack
    padded_adj = np.array([pad_array(m, (max_atoms_limit, max_atoms_limit)) for m in all_adj_matrices])
    padded_feat = np.array([pad_array(f, (max_atoms_limit, atom_feat_dim)) for f in all_atom_features])
    padded_simg = np.array([pad_array(sg, (max_atoms_limit, max_atoms_limit)) for sg in all_sim_graphs])

    np.save(os.path.join(output_npdata_dir, "adj.npy"), padded_adj)
    np.save(os.path.join(output_npdata_dir, "feature.npy"), padded_feat)
    np.save(os.path.join(output_npdata_dir, "graph.npy"), padded_simg) # Save sim_graph as graph.npy

    print(f"Saved featurizer outputs for {len(successfully_processed_indices)} molecules in {output_npdata_dir}")
    return successfully_processed_indices


# %%
# --- Main execution block for Lifespan data ---
def main_featurizer_lifespan(
    base_dir: str, # Base project directory e.g., D:\FJTCM\DeepLife\DLProject
    max_atoms_for_padding: int,
    atom_feature_dimension: int
    ):

    output_features_base = os.path.join(base_dir, "processed_graph_features") # Subdir for these features

    # --- Process training data ---
    train_csv_file = os.path.join(base_dir, "train.csv")
    train_target_name = "LifespanReg_train" # Consistent with Notebook CONFIG
    print(f"\n--- Processing Training Data for Graph Features: {train_target_name} ---")
    if not os.path.exists(train_csv_file):
        print(f"ERROR: {train_csv_file} not found!")
        return

    df_train = pd.read_csv(train_csv_file)
    if "SMILES" not in df_train.columns or "Life_extended" not in df_train.columns:
        print("ERROR: train.csv must contain 'SMILES' and 'Life_extended' columns.")
        return

    print("Cleaning training SMILES...")
    # Pass the Series with its original index to clean_smiles
    train_smiles_series_cleaned = clean_smiles(df_train["SMILES"].copy())
    print(f"  Original train SMILES: {len(df_train)}, Cleaned train SMILES: {len(train_smiles_series_cleaned)}")

    if not train_smiles_series_cleaned.empty:
        train_npdata_output_dir = os.path.join(output_features_base, "npdata", train_target_name)
        # Featurize and save adj.npy, feature.npy, graph.npy
        successful_train_indices = featurize_and_save(
            smiles_series=train_smiles_series_cleaned, # Pass cleaned SMILES
            target_name_suffix=train_target_name,
            output_npdata_dir=train_npdata_output_dir,
            max_atoms_limit=max_atoms_for_padding,
            atom_feat_dim=atom_feature_dimension
        )

        # Save NUMERICAL labels corresponding to successfully featurized SMILES
        if successful_train_indices:
            # Get the original 'Life_extended' values for the successfully processed SMILES
            aligned_labels = df_train.loc[successful_train_indices, "Life_extended"].values.astype(np.float32)
            label_output_path = os.path.join(train_npdata_output_dir, "label.npy")
            np.save(label_output_path, aligned_labels)
            print(f"  Saved label.npy with {len(aligned_labels)} entries to {label_output_path}")
        else:
            print("  No SMILES successfully featurized for training set, so no labels saved.")
    else:
        print("  No valid SMILES after cleaning for training set. Skipping featurization.")


    # --- Process test data ---
    test_csv_file = os.path.join(base_dir, "test.csv")
    test_target_name = "LifespanReg_test" # Consistent with Notebook CONFIG
    print(f"\n--- Processing Test Data for Graph Features: {test_target_name} ---")
    if not os.path.exists(test_csv_file):
        print(f"ERROR: {test_csv_file} not found!")
        return

    df_test = pd.read_csv(test_csv_file)
    if "SMILES" not in df_test.columns:
        print("ERROR: test.csv must contain 'SMILES' column.")
        return

    print("Cleaning test SMILES...")
    test_smiles_series_cleaned = clean_smiles(df_test["SMILES"].copy())
    print(f"  Original test SMILES: {len(df_test)}, Cleaned test SMILES: {len(test_smiles_series_cleaned)}")

    if not test_smiles_series_cleaned.empty:
        test_npdata_output_dir = os.path.join(output_features_base, "npdata", test_target_name)
        # Featurize and save adj.npy, feature.npy, graph.npy
        _ = featurize_and_save( # We don't need successful_indices for test here
            smiles_series=test_smiles_series_cleaned,
            target_name_suffix=test_target_name,
            output_npdata_dir=test_npdata_output_dir,
            max_atoms_limit=max_atoms_for_padding,
            atom_feat_dim=atom_feature_dimension
        )
        # No labels for test set
    else:
        print("  No valid SMILES after cleaning for test set. Skipping featurization.")

    print(f"\nGraph feature processing complete. Check the '{output_features_base}' directory.")


if __name__ == "__main__":
    PROJECT_BASE_DIR = r"D:\FJTCM\DeepLife\DLProject" # Raw string for Windows paths
    
    # These should match the CONFIG in your Jupyter Notebook
    MAX_ATOMS_PAD = 200  # From original featurizer.py example, adjust as needed
    ATOM_FEAT_DIM = 75 # Default for ConvMolFeaturizer

    main_featurizer_lifespan(
        base_dir=PROJECT_BASE_DIR,
        max_atoms_for_padding=MAX_ATOMS_PAD,
        atom_feature_dimension=ATOM_FEAT_DIM
    )



--- Processing Training Data for Graph Features: LifespanReg_train ---
Cleaning training SMILES...
  Original train SMILES: 1679, Cleaned train SMILES: 1679
Starting featurization for: LifespanReg_train


[22:46:17] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:17] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:17] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:17] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:17] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:17] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:17] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:17] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:17] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:17] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:17] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:17] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:17] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:17] DEPRECATION WARNING: please use GetValen

Saved featurizer outputs for 1679 molecules in D:\FJTCM\DeepLife\DLProject\processed_graph_features\npdata\LifespanReg_train
  Saved label.npy with 1679 entries to D:\FJTCM\DeepLife\DLProject\processed_graph_features\npdata\LifespanReg_train\label.npy

--- Processing Test Data for Graph Features: LifespanReg_test ---
Cleaning test SMILES...
  Original test SMILES: 21, Cleaned test SMILES: 21
Starting featurization for: LifespanReg_test
Saved featurizer outputs for 21 molecules in D:\FJTCM\DeepLife\DLProject\processed_graph_features\npdata\LifespanReg_test

Graph feature processing complete. Check the 'D:\FJTCM\DeepLife\DLProject\processed_graph_features' directory.


[22:46:23] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:23] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:23] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:23] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:23] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:23] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:23] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:23] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:23] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:23] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:23] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:23] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:23] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[22:46:23] DEPRECATION WARNING: please use GetValen